In [ ]:
def append_to_sys_path(folder: str, base_folder: str = "weather_bundle") -> str:
    """
    Append the specified folder to the system path if it is not already present.
    Allows importing packages from the folder.
    """
    import re
    import os
    import sys

    if not (match := re.search(rf"(.+[\\/]{base_folder})", os.getcwd())):
        raise FileNotFoundError(f"Could not find {base_folder} in current path: {os.getcwd()}")

    base_path = match.group(1)
    for root, dirs, _ in os.walk(base_path):
        if "src" in dirs:
            base_path = root
            break

    if (path := os.path.join(base_path, folder)) not in sys.path:
        sys.path.append(path)
    return path


append_to_sys_path("src/config")

In [ ]:
from config import Config
import os

config = Config()
catalog = config["weather"]["catalog_name"]
bronze_schema = config["weather"]["bronze_schema_name"]
noaa: dict = config["weather"]["noaa"]
bronze_metrics_checkpoint_folder = noaa["metrics"]["bronze_metrics_checkpoint_folder"]

checkpoint_path = os.path.join(
    "/Volumes", catalog, bronze_schema, bronze_metrics_checkpoint_folder
)

print(f"{checkpoint_path=}")

In [ ]:
# Remove all records from the table
dbutils.fs.rm(checkpoint_path, recurse=True)

In [ ]:
# Check if path was removed
print(f"Contents of {checkpoint_path}:")
try:
    contents = dbutils.fs.ls(checkpoint_path)
    if contents:
        print("Directory was NOT removed. Contents:")
        for item in contents:
            print(item)
    else:
        print("Directory is empty.")
except Exception as e:
    print("Directory does not exist.")